In [6]:
import glob
import os
import yaml
import pandas as pd


In [2]:
def open_sample_sheet(sample_sheet_fp, lanes=False, skiprows=18):
    """Read in an IGM sample sheet and return a pandas DF with primary data table"""
    sample_sheet = pd.read_excel(sample_sheet_fp, skiprows = skiprows, header=1)

    if lanes:
        sample_sheet = sample_sheet.loc[sample_sheet['Lane'].isin(lanes)]

    return(sample_sheet)

In [3]:
def open_sequencing_manifest(manifest_fp, lanes='False', sheetname='Sample Information', skiprows=18):
    """Read in an IGM sequencing manifest and return a pandas DF with primary data table"""
    sample_sheet = pd.read_excel(manifest_fp, sheetname=sheetname, skiprows=skiprows, header=1)
    
    if lanes:
        sample_sheet = sample_sheet.loc[sample_sheet['lane'].isin(lanes)]
        
    return(sample_sheet)

In [4]:
def get_read(sample, seq_dir, read):
    """Function to pull a given read based on sample name from the reads directory"""
    reads = glob.glob(os.path.join(seq_dir, "{0}_*_{1}_*.fastq.gz".format(sample, read)))
    if len(reads) == 1:
        return(reads[0])
    elif len(reads) > 1:
        raise ValueError('Too many reads found for {0} in {1}:\n'
                         'read_str: {2}'.format(sample, seq_dir, reads))
    elif len(reads) < 1:
        raise ValueError('Too few reads found for {0} in {1}:\n'
                         'read_str: {1}\n'.format(sample, seq_dir, reads))   

In [17]:
def make_sample_dict_pe(sample_sheet, seq_dir,
                        forward = 'R1',
                        reverse = 'R2',
                        adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                        phred = 'phred33',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample',
                        tolerate_losses = True):
    
    samples_pe = {'samples_pe': {}}
    for x in sample_sheet.index:
        try:
            samples_pe['samples_pe'][sample_sheet.loc[x, sample_name]] = {
               'forward': get_read(sample_sheet.loc[x, sample_header], seq_dir, forward),
               'reverse': get_read(sample_sheet.loc[x, sample_header], seq_dir, reverse),
               'adaptor': adaptor,
               'phred': phred
            }
        except ValueError:
            if tolerate_losses:
                print('No sequences found for %s' % sample_sheet.loc[x, sample_name])
            else:
                raise ValueError
    
    return(samples_pe)

In [18]:
def make_sample_dict_se(sample_sheet, seq_dir,
                        forward = 'R1',
                        adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                        phred = 'phred33',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample',
                        tolerate_losses = True):
    
    samples_pe = {'samples_pe': {}}
    for x in sample_sheet.index:
        try:
            samples_pe['samples_pe'][sample_sheet.loc[x, sample_name]] = {
               'forward': get_read(sample_sheet.loc[x, sample_header], seq_dir, forward),
               'adaptor': adaptor,
               'phred': phred
            }
        except ValueError:
            if tolerate_losses:
                print('No sequences found for %s' % sample_sheet.loc[x, sample_name])
            else:
                raise ValueError
    
    return(samples_pe)

In [19]:
def format_yaml_pe(RUN, samples_pe,
                   TMP_DIR_ROOT = '/localscratch',
                   trimmomatic_path = '$CONDA_ENV_PATH/share/trimmomatic-*/trimmomatic',
                   adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                   gzip_path = 'gzip',
                   knead_env = 'source activate kneaddata',
                   bowtie_env = 'source activate kneaddata',
                   qc_env = 'source activate kneaddata',
                   trim_env = 'source activate kneaddata',
                   host_db = '/projects/mammal_glycans/haddad_mice/dbs/mouse/mouse',
                   params = {'trimmomatic_params': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33'},
                   default_flow_style = False):
    
    config_str = ''

    config_str += yaml.dump({'TMP_DIR_ROOT': TMP_DIR_ROOT}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'RUN': RUN}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'KNEAD_ENV': knead_env}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'BOWTIE_ENV': bowtie_env}, default_flow_style = default_flow_style)
        
    config_str += yaml.dump({'QC_ENV': qc_env}, default_flow_style = default_flow_style)
        
    config_str += yaml.dump({'TRIM_ENV': trim_env}, default_flow_style = default_flow_style)
            
    config_str += yaml.dump({'HOST_DB': host_db}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump(samples_pe, default_flow_style = default_flow_style)

    config_str += yaml.dump({'software': 
                   {'trimmomatic': trimmomatic_path,
                    'gzip': gzip_path}
              }, default_flow_style = default_flow_style)

    config_str += yaml.dump(params,
                            default_flow_style = default_flow_style)

    return(config_str)

In [20]:
def format_yaml_se(RUN, samples_se,
                   TMP_DIR_ROOT = '/localscratch',
                   trimmomatic_path = '$CONDA_ENV_PATH/share/trimmomatic-*',
                   adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                   kneaddata_db = '$CONDA_ENV_PATH/share/kd_dbs/demo',
                   gzip_path = 'gzip',
                   knead_env = 'source activate kneaddata',
                   params = {'trimmomatic_params': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33'},
                   default_flow_style = False):
    
    config_str = ''

    config_str += yaml.dump({'TMP_DIR_ROOT': TMP_DIR_ROOT}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'RUN': RUN}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'KNEAD_ENV': knead_env}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'kneaddata_db': kneaddata_db}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump(samples_se, default_flow_style = default_flow_style)

    config_str += yaml.dump({'software': 
                   {'trimmomatic': trimmomatic_path,
                    'gzip': gzip_path}
              }, default_flow_style = default_flow_style)

    config_str += yaml.dump(params,
                            default_flow_style = default_flow_style)

    return(config_str)

# Read in the sample data 

In [11]:
# this is the Excel document provided by IGM
sequencing_manifest_fp = './example/reads/example_sample_sheet.xls'

# If you only want samples from some lanes, you can provide a list of those lane numbers here
lanes = []

# This is the path to the folder on Barnacle where the raw reads are located
seq_dir = './example/reads/Run1/'

# Read in the sample manifest 
sample_sheet = open_sample_sheet(sequencing_manifest_fp, lanes=lanes)

# These columns can be modified with regexes to fix sample names or add common sample prefixes if necessary
# sample_sheet['Sample_Prefix'] = sample_sheet['Sample Name'].replace('\.','_', regex=True)
# sample_sheet['Sample'] = sample_sheet['Sample Name'].replace('\.','_', regex=True)
sample_sheet['Sample_Prefix'] = sample_sheet['Sample_Name']
sample_sheet['Sample'] = sample_sheet['Sample_Name']

sample_sheet

,Lane,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,Sample_Project,Description,Sample_Prefix,Sample
0,1,sample1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,sample2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Make samples dictionary
------

This links the sample name to the original location of the forward (and reverse) reads. 

To instead use single ended reads, cal `samples_se()` instead.

In [15]:
samples_pe = make_sample_dict_pe(sample_sheet, seq_dir,
                        adaptor = '/home/jgsanders/git_sw/git_bin/Trimmomatic-0.36/adapters/TruSeq3-PE-2.fa',
                        forward = 'R1',
                        reverse = 'R2',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample')

No sequences found for nan
No sequences found for nan


Format YAML file
================

Make the actual config file.

Going to want to modify `RUN` variable with a good name for the run.

You may also want to pass optional parameters to modify execution. The optional parameters and their defaults are:

```
TMP_DIR_ROOT = '/localscratch',
trimmomatic_path = '$CONDA_ENV_PATH/share/trimmomatic-*',
adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
kneaddata_db = '$CONDA_ENV_PATH/share/kd_dbs/demo',
gzip_path = 'gzip',
knead_env = 'source activate kneaddata',
params = {'trimmomatic_params': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33'}
```

In [21]:
RUN = 'Run1'
config_str = format_yaml_pe(RUN, samples_pe)
with open('config_%s.yaml' % RUN, 'w') as f:
    f.write(config_str)

In [22]:
print(config_str)

TMP_DIR_ROOT: /localscratch
RUN: Run1
KNEAD_ENV: source activate kneaddata
BOWTIE_ENV: source activate kneaddata
QC_ENV: source activate kneaddata
TRIM_ENV: source activate kneaddata
HOST_DB: /projects/mammal_glycans/haddad_mice/dbs/mouse/mouse
samples_pe: {}
software:
  gzip: gzip
  trimmomatic: $CONDA_ENV_PATH/share/trimmomatic-*/trimmomatic
trimmomatic_params: LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33

